In [1]:
import pandas as pd
import numpy as np 
import rdkit
from rdkit import Chem 
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

pd.__version__, np.__version__, rdkit.__version__, torch.__version__, torch.cuda.is_available(), torch_geometric.__version__

('1.4.4', '1.23.1', '2022.03.2', '1.11.0+cu113', True, '2.1.0')

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if device == 'cuda:0':
    torch.cuda.manual_seed_all(seed)
    
device

device(type='cuda', index=0)

In [3]:
import os
from tqdm import tqdm

In [4]:
class MoleculeDataset(Dataset): #Dataset에서 상속 받음
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return self.filename

    @property
    def processed_file_names(self): #파일 존재 여부 확인
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        if self.test:
            return [f'testset_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'trainset_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, sequences in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol = Chem.MolFromSmiles(
                '.'.join( #.는 non-bond
                    Chem.MolToSmiles(
                        Chem.MolFromSequence(sequences[sequence])
                    ) for sequence in ['mhc_sequence', 'sequence']
                )
            )
            mol = Chem.AddHs(mol)
            x = self._get_node_features(mol)
            edge_attr, edge_index, edge_weight = self._get_edge_features(mol)
            y = self._get_labels(sequences['binding']) 
            data = Data(x=x, edge_attr=edge_attr, edge_index=edge_index, edge_weight=edge_weight, y=y)
            
            if self.test:
                torch.save(data, os.path.join(self.processed_dir, f'testset_{index}.pt'))
            else:
                torch.save(data, os.path.join(self.processed_dir, f'trainset_{index}.pt'))

    def _get_node_features(self, mol):
        all_node_feats = []

        for atom in mol.GetAtoms(): #원자에 대해
            symbol = atom.GetSymbol() #원소기호
            valid_atoms = {'H': 0, 'C':1, 'N':2, 'O':3, 'S':4}
            atom_one_hot = [0] * len(valid_atoms)
            try:
                idx = valid_atoms[symbol]
                atom_one_hot[idx] = 1
            except:
                pass

            hybrid = atom.GetHybridization()
            hybrid_one_hot = [0] * 7
            if hybrid == Chem.HybridizationType.SP3:
                hybrid_one_hot[0] = 1
            elif hybrid == Chem.HybridizationType.SP2:
                hybrid_one_hot[1] = 1
            elif hybrid == Chem.HybridizationType.SP:
                hybrid_one_hot[2] = 1
            elif hybrid == Chem.HybridizationType.S:
                hybrid_one_hot[3] = 1
            elif hybrid == Chem.HybridizationType.SP3D:
                hybrid_one_hot[4] = 1
            elif hybrid == Chem.HybridizationType.SP3D2:
                hybrid_one_hot[5] = 1
            else:
                hybrid_one_hot[6] = 1

            degree = atom.GetDegree() #공유결합의 개수
            degree_one_hot = [0, 0, 0, 0, 0, 0]
            if degree >= 5: # 5개 이상의 공유 결합을 가지는 원자.
                degree_one_hot[5]=1
            else:
                degree_one_hot[degree]=1

            num_h = atom.GetTotalNumHs() #결합된 수소원자의 개수
            h_one_hot = [0, 0, 0, 0, 0]
            if num_h >= 4:
                h_one_hot[4] = 1
            else:
                h_one_hot[num_h] = 1

            chiral = atom.GetChiralTag() #손대칭성
            if chiral == Chem.rdchem.ChiralType.CHI_OTHER:
                chiral_one_hot = [1, 0, 0, 0]
            elif chiral == Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW:
                chiral_one_hot = [0, 1, 0, 0]
            elif chiral == Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW:
                chiral_one_hot = [0, 0, 1, 0]
            elif chiral == Chem.rdchem.ChiralType.CHI_UNSPECIFIED:
                chiral_one_hot = [0, 0, 0, 1]

            aromatic = 1 if atom.GetIsAromatic() else 0 #방향족 여부
            ring = 1 if atom.IsInRing() else 0 #고리 내 포함 여부

            node_attr = atom_one_hot + hybrid_one_hot + degree_one_hot + h_one_hot + chiral_one_hot + [aromatic, ring, atom.GetFormalCharge(), atom.GetNumRadicalElectrons()]
            all_node_feats.append(node_attr)
            
        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        all_edge_feats = []
        edge_indices = []
        edge_weights = []
        for bond in mol.GetBonds(): #공유결합에 대해
            bond_type = bond.GetBondType() #공유 결합의 종류
            if bond_type == Chem.rdchem.BondType.SINGLE:
                bond_one_hot = [1, 0, 0, 0]
                edge_weights.extend([1.0, 1.0])
            elif bond_type == Chem.rdchem.BondType.AROMATIC:
                bond_one_hot = [0, 1, 0, 0]
                edge_weights.extend([1.5, 1.5])
            elif bond_type == Chem.rdchem.BondType.DOUBLE:
                bond_one_hot = [0, 0, 1, 0]
                edge_weights.extend([2.0, 2.0])
            elif bond_type == Chem.rdchem.BondType.TRIPLE:
                bond_one_hot = [0, 0, 0, 1]   
                edge_weights.extend([3.0, 3.0])  

            stereo_type = bond.GetStereo()
            if stereo_type == Chem.rdchem.BondStereo.STEREOANY:
                stereo_one_hot = [1, 0, 0, 0, 0, 0]
            elif stereo_type == Chem.rdchem.BondStereo.STEREOCIS:
                stereo_one_hot = [0, 1, 0, 0, 0, 0]
            elif stereo_type == Chem.rdchem.BondStereo.STEREOE:
                stereo_one_hot = [0, 0, 1, 0, 0, 0]
            elif stereo_type == Chem.rdchem.BondStereo.STEREONONE:
                stereo_one_hot = [0, 0, 0, 1, 0, 0]
            elif stereo_type == Chem.rdchem.BondStereo.STEREOTRANS:
                stereo_one_hot = [0, 0, 0, 0, 1, 0]
            elif stereo_type == Chem.rdchem.BondStereo.STEREOZ:
                stereo_one_hot = [0, 0, 0, 0, 0, 1]

            ring_bond = 1 if bond.IsInRing() else 0 #링 내 포함 여부
            conjugate_bond = 1 if bond.GetIsConjugated() else 0

            edge_attr = bond_one_hot + stereo_one_hot + [ring_bond, conjugate_bond]
            all_edge_feats.append(edge_attr) #무향 그래프이므로 두번 추가
            all_edge_feats.append(edge_attr)
            
            i = bond.GetBeginAtomIdx() #시작 원자의 인덱스
            j = bond.GetEndAtomIdx() #종점 원자의 인덱스
            edge_indices += [[i, j], [j, i]]

        all_edge_feats = np.asarray(all_edge_feats)
        all_edge_feats = torch.tensor(all_edge_feats, dtype=torch.float)
        
        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        
        edge_weights = torch.tensor(edge_weights, dtype = torch.float)
        
        return all_edge_feats, edge_indices, edge_weights

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, f'testset_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, f'trainset_{idx}.pt'))   
        return data

In [5]:
trainset = MoleculeDataset(root='iedb_sequences', filename='iedb_trainset.csv')
trainset

MoleculeDataset(125667)

In [6]:
trainset[0]

Data(x=[696, 31], edge_index=[2, 1416], edge_attr=[1416, 12], y=[1], edge_weight=[1416])

In [7]:
testset = MoleculeDataset(root='iedb_sequences', filename='iedb_testset.csv', test=True)
testset

MoleculeDataset(31417)

In [8]:
testset[0]

Data(x=[721, 31], edge_index=[2, 1478], edge_attr=[1478, 12], y=[1], edge_weight=[1478])

In [9]:
batch_size = 64
#batch_size = 128
#batch_size = 256
#batch_size = 512
#batch_size = 1024
#batch_size = 2048
#batch_size = 4096

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, drop_last=False)

In [10]:
import torch
import torch.nn.functional as F #활성함수나 드랍아웃을 사용하기 위해
from torch.nn import MultiheadAttention, Conv1d, BatchNorm1d, MaxPool1d, Conv2d, BatchNorm2d, MaxPool2d, AvgPool1d
from torch.nn import Linear, ModuleList #모듈리스트를 정의하는 이유는 학습을 할 수 있도록 하기 위해
from torch_geometric.nn import GATConv, TransformerConv, SAGPooling, TopKPooling, GraphNorm, BatchNorm
from torch_geometric.nn import global_mean_pool as gmp #노드 특징 벡터들의 평균값. 그래프 행렬를 벡터로 바꿔줌 (permutation invariance하게). 리드아웃에 사용. 리드아웃은 그래프 분류에만 사용되며 모든 노드의 은닉 상태를 더하여 그래프의 은닉 상태를 만드는 과정
#from torch_geometric.nn import global_max_pool as gmp #노드 특징 벡터들 중 최대값

#일반적인 DCNN의 적층 순서: Conv > BN > Activation > Dropout > Pooling

class GNN(torch.nn.Module): #nn.Modlule 상속
    def __init__(self, hyperparameters):
        super(GNN, self).__init__()
        in_channels= hyperparameters["in_channels"]
        channels= hyperparameters["channels"]
        heads = hyperparameters["heads"]
        self.heads = heads
        dropout_rate = hyperparameters["dropout_rate"]
        self.dropout_rate = dropout_rate
        edge_dim = hyperparameters["edge_dim"]
        kernel_size = hyperparameters["kernel_size"]

        self.conv1 = GATConv(in_channels, channels, heads=heads, dropout=dropout_rate, edge_dim=edge_dim, concat=False) 
        self.norm1 = GraphNorm(channels)
        
        self.conv2 = GATConv(channels, channels, heads=heads, dropout=dropout_rate, edge_dim=edge_dim, concat=False) 
        self.norm2 = GraphNorm(channels)
        
        self.conv3 = GATConv(channels, channels, heads=heads, dropout=dropout_rate, edge_dim=edge_dim, concat=False) 
        self.norm3 = GraphNorm(channels)
        
        self.conv4 = GATConv(channels, channels, heads=heads, dropout=dropout_rate, edge_dim=edge_dim, concat=False) 
        self.norm4 = GraphNorm(channels)
        
        self.conv5 = Conv1d(heads, heads*2, kernel_size=kernel_size, padding='same')
        self.norm5 = BatchNorm1d(heads*2)
        self.pool5 = AvgPool1d(2)
        
        self.conv6 = Conv1d(heads*2, heads*4, kernel_size=kernel_size, padding='same')
        self.norm6 = BatchNorm1d(heads*4)
        self.pool6 = AvgPool1d(2)
        
        self.lin7 = Linear(channels, 1)
        
    def forward(self, x, edge_attr, edge_index, edge_weight, batch_index):
        x = self.conv1(x, edge_index, edge_attr)
        x = self.norm1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=self.dropout_rate)
        
        x = self.conv2(x, edge_index, edge_attr)
        x = self.norm2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=self.dropout_rate)
        
        x = self.conv3(x, edge_index, edge_attr)
        x = self.norm3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=self.dropout_rate)
        
        x = self.conv4(x, edge_index, edge_attr)
        x = self.norm4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=self.dropout_rate)
        
        x = gmp(x, batch_index)
        skip4 = x
        x = torch.reshape(x, (x.shape[0], self.heads, x.shape[1]//self.heads)) #헤드 수만큼 채널에 넣음
        
        x = self.conv5(x)
        x = self.norm5(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=self.dropout_rate)
        x = self.pool5(x)
        skip5 = x.flatten(start_dim=1, end_dim=2)
                
        x = self.conv6(x)
        x = self.norm6(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=self.dropout_rate)
        x = self.pool6(x)
        
        x = x.flatten(start_dim=1, end_dim=2)
        x += skip4 + skip5 #skip connection
        
        x = self.lin7(x)
#         x = torch.sigmoid(x)
        
        return x

In [11]:
from sklearn import metrics

def evaluate(labels, predictions, mode):
    roc_auc = metrics.roc_auc_score(labels, predictions)
    print(mode, 'ROC AUC:', roc_auc)
    fpr, tpr, thresholds = metrics.roc_curve(labels, predictions)
    optimal_threshold = thresholds[np.argmax(tpr-fpr)]
    casted = np.array([1 if element >= optimal_threshold else 0 for element in predictions])
    print(mode, 'F1-score:', metrics.f1_score(labels, casted))
    print(mode, 'Precision:', metrics.precision_score(labels, casted))
    print(mode, 'Recall:', metrics.recall_score(labels, casted))
    precision, recall, thresholds = metrics.precision_recall_curve(labels, predictions)
    pr_auc = metrics.auc(recall, precision)
    print(mode, 'PR AUC', pr_auc)
    print(mode, 'Accuracy:', metrics.accuracy_score(labels, casted))
    print(mode, 'Optimal threshold:', optimal_threshold)
    return f'{roc_auc:.3f}'

In [12]:
from datetime import datetime

def train(epoch, model, train_loader, optimizer, criterion):
    model.train() #학습 모드
    labels = []
    predictions = []
    first = False
    for batch in tqdm(train_loader):
        batch.to(device)  
        labels.append(batch.y.float().cpu().detach().numpy())
        optimizer.zero_grad() #기울기 초기화
        prediction = model(batch.x.float(), batch.edge_attr.float(), batch.edge_index, batch.edge_weight, batch.batch,) 
        loss = criterion(torch.squeeze(prediction), batch.y.float())
        loss.backward() #미분, 역전파
        optimizer.step() #가중치 갱신
        predictions.append(torch.sigmoid(prediction).cpu().detach().numpy()) #시그모이드 적용
    labels = np.concatenate(labels).ravel()
    predictions = np.concatenate(predictions).ravel()
    evaluate(labels, predictions, 'Train,')

def test(epoch, model, test_loader, criterion):
    model.eval() #추론 모드
    labels = []
    predictions = []
    for batch in test_loader:  
        batch.to(device) 
        labels.append(batch.y.float().cpu().detach().numpy())
        prediction = model(batch.x.float(), batch.edge_attr.float(), batch.edge_index, batch.edge_weight, batch.batch,)
        predictions.append(torch.sigmoid(prediction).cpu().detach().numpy()) #시그모이드 적용
    labels = np.concatenate(labels).ravel()
    predictions = np.concatenate(predictions).ravel()
    return evaluate(labels, predictions, 'Test,')

In [13]:
hyperparameters = {
    "in_channels": trainset[0].x.shape[1],
    "channels": 256,
    "heads": 8,
    "dropout_rate": 0.1,
    "edge_dim": trainset[0].edge_attr.shape[1],
    "kernel_size": 9,
}

model = GNN(hyperparameters=hyperparameters)
model = model.to(device)
print(model)

GNN(
  (conv1): GATConv(31, 256, heads=8)
  (norm1): GraphNorm(256)
  (conv2): GATConv(256, 256, heads=8)
  (norm2): GraphNorm(256)
  (conv3): GATConv(256, 256, heads=8)
  (norm3): GraphNorm(256)
  (conv4): GATConv(256, 256, heads=8)
  (norm4): GraphNorm(256)
  (conv5): Conv1d(8, 16, kernel_size=(9,), stride=(1,), padding=same)
  (norm5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool5): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv6): Conv1d(16, 32, kernel_size=(9,), stride=(1,), padding=same)
  (norm6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool6): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (lin7): Linear(in_features=256, out_features=1, bias=True)
)


In [14]:
parameters = 0
for name, parameter in model.named_parameters():
    if parameter.requires_grad:
        print(name, parameter.numel())
        parameters += parameter.numel()
parameters

conv1.att_src 2048
conv1.att_dst 2048
conv1.att_edge 2048
conv1.bias 256
conv1.lin_src.weight 63488
conv1.lin_edge.weight 24576
norm1.weight 256
norm1.bias 256
norm1.mean_scale 256
conv2.att_src 2048
conv2.att_dst 2048
conv2.att_edge 2048
conv2.bias 256
conv2.lin_src.weight 524288
conv2.lin_edge.weight 24576
norm2.weight 256
norm2.bias 256
norm2.mean_scale 256
conv3.att_src 2048
conv3.att_dst 2048
conv3.att_edge 2048
conv3.bias 256
conv3.lin_src.weight 524288
conv3.lin_edge.weight 24576
norm3.weight 256
norm3.bias 256
norm3.mean_scale 256
conv4.att_src 2048
conv4.att_dst 2048
conv4.att_edge 2048
conv4.bias 256
conv4.lin_src.weight 524288
conv4.lin_edge.weight 24576
norm4.weight 256
norm4.bias 256
norm4.mean_scale 256
conv5.weight 1152
conv5.bias 16
norm5.weight 16
norm5.bias 16
conv6.weight 4608
conv6.bias 32
norm6.weight 32
norm6.bias 32
lin7.weight 256
lin7.bias 1


1769489

In [15]:
trainset_raw = pd.read_csv('iedb_sequences/raw/iedb_trainset.csv')
pos_weight = (trainset_raw['binding']==0).sum()/trainset_raw['binding'].sum()
pos_weight = torch.tensor([pos_weight], dtype=torch.float).to(device)
pos_weight

tensor([2.8409], device='cuda:0')

In [16]:
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight) #로짓이 입혀진 바이너리 크로스 엔트로피 로스

for epoch in range(100): 
    print('Epoch', epoch)
    train(epoch, model, train_loader, optimizer, criterion)
    with torch.no_grad():
        roc_auc = test(epoch, model, test_loader, criterion)
    now = datetime.now().strftime('%Y%m%dT%H%M%S')
    filename = f'{now}_model_{roc_auc}.pt'
    torch.save(model, filename)
    print()

Epoch 0


100%|██████████| 1964/1964 [08:27<00:00,  3.87it/s]


Train, ROC AUC: 0.6807808245495387
Train, F1-score: 0.46805277184327027
Train, Precision: 0.3658414818895469
Train, Recall: 0.6495201418179596
Train, PR AUC 0.434656186199152
Train, Accuracy: 0.6156190567133775
Train, Optimal threshold: 0.49284852
Test, ROC AUC: 0.7180876216154006
Test, F1-score: 0.49437673521246067
Test, Precision: 0.40206659012629165
Test, Recall: 0.6417053505985829
Test, PR AUC 0.4867464659242218
Test, Accuracy: 0.6579877136582105
Test, Optimal threshold: 0.62458026

Epoch 1


100%|██████████| 1964/1964 [08:27<00:00,  3.87it/s]


Train, ROC AUC: 0.7329544354803288
Train, F1-score: 0.5101158179714442
Train, Precision: 0.4032788049945827
Train, Recall: 0.6939605110336817
Train, PR AUC 0.49885667866540084
Train, Accuracy: 0.6529797003190972
Train, Optimal threshold: 0.4957959
Test, ROC AUC: 0.7560193365094071
Test, F1-score: 0.529635823114084
Test, Precision: 0.427297256783091
Test, Recall: 0.6964329342780357
Test, PR AUC 0.5355362302086288
Test, Accuracy: 0.6776904223827864
Test, Optimal threshold: 0.4303698

Epoch 2


...

Train, ROC AUC: 0.9274045234319341
Train, F1-score: 0.7411376751854905
Train, Precision: 0.6405129803624705
Train, Recall: 0.8792713491044685
Train, PR AUC 0.8097278048891516
Train, Accuracy: 0.8400853048135151
Train, Optimal threshold: 0.4933977
Test, ROC AUC: 0.9208699515145915
Test, F1-score: 0.7371862476270828
Test, Precision: 0.6485433290035257
Test, Recall: 0.853896897141461
Test, PR AUC 0.8022866179682645
Test, Accuracy: 0.8413597733711048
Test, Optimal threshold: 0.45089594

Epoch 98


100%|██████████| 1964/1964 [08:26<00:00,  3.88it/s]


Train, ROC AUC: 0.927188790526431
Train, F1-score: 0.7374922005322738
Train, Precision: 0.6320913277890555
Train, Recall: 0.8850785500336207
Train, PR AUC 0.8118129980698859
Train, Accuracy: 0.8359553422935218
Train, Optimal threshold: 0.47533017
Test, ROC AUC: 0.9212453992098797
Test, F1-score: 0.7371025479048221
Test, Precision: 0.647641073080481
Test, Recall: 0.8552406547764476
Test, PR AUC 0.8021297078412108
Test, Accuracy: 0.8410414743610147
Test, Optimal threshold: 0.4787522

Epoch 99


100%|██████████| 1964/1964 [08:26<00:00,  3.88it/s]


Train, ROC AUC: 0.9281114071896375
Train, F1-score: 0.743328556489407
Train, Precision: 0.6434654328438275
Train, Recall: 0.8798826334128003
Train, PR AUC 0.811665569209044
Train, Accuracy: 0.8417961756069613
Train, Optimal threshold: 0.49373272
Test, ROC AUC: 0.9221035492194872
Test, F1-score: 0.7321069717112382
Test, Precision: 0.6330986543088851
Test, Recall: 0.8678231126313217
Test, PR AUC 0.8068071022869412
Test, Accuracy: 0.8345163446541681
Test, Optimal threshold: 0.44602984



In [17]:
#pretrained_model = torch.load('models/20221101T013506_model_0.920.pt', map_location=device)
pretrained_model = torch.load(filename, map_location=device)
pretrained_model

GNN(
  (conv1): GATConv(31, 256, heads=8)
  (norm1): GraphNorm(256)
  (conv2): GATConv(256, 256, heads=8)
  (norm2): GraphNorm(256)
  (conv3): GATConv(256, 256, heads=8)
  (norm3): GraphNorm(256)
  (conv4): GATConv(256, 256, heads=8)
  (norm4): GraphNorm(256)
  (conv5): Conv1d(8, 16, kernel_size=(9,), stride=(1,), padding=same)
  (norm5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool5): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv6): Conv1d(16, 32, kernel_size=(9,), stride=(1,), padding=same)
  (norm6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool6): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (lin7): Linear(in_features=256, out_features=1, bias=True)
)

In [18]:
criterion = torch.nn.BCELoss() #바이너리 크로스 엔트로피 로스

with torch.no_grad():
    test(0, pretrained_model, test_loader, criterion)

Epoch 0
Test, ROC AUC: 0.9218283124071883
Test, F1-score: 0.7315258383447151
Test, Precision: 0.6275795732773697
Test, Recall: 0.8767407769362326
Test, PR AUC 0.8037979538311759
Test, Accuracy: 0.8323200814845466
Test, Optimal threshold: 0.42802235

